In [1]:
import pandas as pd 
import numpy as np

import requests
from bs4 import BeautifulSoup

In [2]:
base_url = 'https://askfred.net/Results/past.php?f%5Bevent_weapon_eq%5D=Epee&f%5Bevent_gender_eq%5D=men&f%5Bevent_age_eq%5D=senior-open&f%5Bradius_mi%5D=300&vals%5Bloc%5D=&f%5Bname_contains%5D=&ops%5Bdate%5D=start_date_eq&vals%5Bdate%5D=&f%5Bevent_is_team%5D=&f%5Bevent_entries_gte%5D=&ops%5Bevent_rating%5D=event_rating_eq&vals%5Bevent_rating%5D=&f%5Bdivision_id%5D=&page_id={}'

In [3]:
response = requests.get(base_url.format(50))
soup = BeautifulSoup(response.text, 'html')

In [4]:
soup.find('h3') is None

True

In [5]:
[t.text.split('\n\n\n')[-1].strip() for t in soup.find('table', {'id': 'past-tours'})]

['',
 'Mar 25 - Mar 27, 2017',
 'Mar 25, 2017',
 'Mar 25 - Mar 26, 2017',
 'Mar 25 - Mar 26, 2017',
 'Mar 24 - Mar 26, 2017',
 'Mar 24, 2017',
 'Mar 23, 2017',
 'Mar 19, 2017',
 'Mar 19, 2017',
 'Mar 19, 2017',
 'Mar 19, 2017',
 'Mar 19, 2017',
 'Mar 19, 2017',
 'Mar 18, 2017',
 'Mar 18, 2017',
 'Mar 17 - Mar 19, 2017',
 'Mar 14, 2017',
 'Mar 12, 2017',
 'Mar 10 - Mar 12, 2017',
 'Mar 5, 2017',
 '']

In [6]:
tournament_links = [a['href'] for a in soup.find('table', {'id': 'past-tours'}).findAll('a') if 'results' in a['href']]
tournament_dates = [t.text.split('\n\n\n')[-1].strip() for t in soup.find('table', {'id': 'past-tours'})]
tournament_dates = [d for d in tournament_dates if d != '']
len(tournament_links), len(tournament_dates)

(14, 20)

In [7]:
tournament_links[12]

'https://askfred.net/Results/results.php?tournament_id=34679'

In [8]:
tournament_links

['https://askfred.net/Results/results.php?tournament_id=33797',
 'https://askfred.net/Results/results.php?tournament_id=35975',
 'https://askfred.net/Results/results.php?tournament_id=33742',
 'https://askfred.net/Results/results.php?tournament_id=36013',
 'https://askfred.net/Results/results.php?tournament_id=34223',
 'https://askfred.net/Results/results.php?tournament_id=36039',
 'https://askfred.net/Results/results.php?tournament_id=36228',
 'https://askfred.net/Results/results.php?tournament_id=36255',
 'https://askfred.net/Results/results.php?tournament_id=36332',
 'https://askfred.net/Results/results.php?tournament_id=34709',
 'https://askfred.net/Results/results.php?tournament_id=36365',
 'https://askfred.net/Results/results.php?tournament_id=33956',
 'https://askfred.net/Results/results.php?tournament_id=34679',
 'https://askfred.net/Results/results.php?tournament_id=35786']

In [37]:
response = requests.get(tournament_links[0])
soup = BeautifulSoup(response.text, 'html')

In [38]:
soup.findAll('table', {'class': 'box'})[0].findAll('tr')[0].text

"\n\nCadet (U16) Men's Epee:\xa0\xa0\xa0\n11 Competitors, a NR Event\n\xa0\xa0\xa0\xa0\xa0\xa0\n\n\n"

In [55]:
def get_event_type(row_text):
    category, _, _, n_rating = row_text.split('\xa0')
    category = category.replace(':', '')
    n, rating = n_rating.split(', ')
    n = int(n.replace('Competitors', ''))
    rating = rating.replace('a ', '').replace(' Event', '')

    return category, n, rating

def get_event_info(tournament_page):
    date, location = tournament_page.find('div', {'id': 'results-head'}).text.split('\n')[1:3]
    return date, location

In [63]:
soup.find('div', {'id': 'results-head'}).text.split('\n')[1:3]

['03/25/2017 - 03/27/2017', 'Arapahoe County Fairgrounds: Aurora, CO']

In [58]:
rows = [r.text.strip() for t in soup.findAll('table', {'class': 'box'}) for r in t.findAll('tr')]
# table_starts = ['Competitors' in r for r in rows]
category, n, rating = None, None, None
data = []
for r in rows:
    if 'Competitors' in r:
        category, n, rating = get_event_type(r)
    
    if 'Rating Earned' in r:
        continue
    
    row_data = [t.strip() for t in rows[2].split('\n') if t != '']
    if len(row_data) < 6:
        row_data += ['']
    data.append([category, n, rating] + row_data)

pd.DataFrame(data, columns=['Event', 'NumCompetitors', 'EventRating', 'Place', 'Name', 'ClubAbbr', 'Club', 'Rating', 'RatingEarned'])

,Event,NumCompetitors,EventRating,Place,Name,ClubAbbr,Club,Rating,RatingEarned
0,Cadet (U16) Men's Epee,11,NR,1,"Senseman, Samuel",NCF,Northern Colorado Fencers,B2016,
1,Cadet (U16) Men's Epee,11,NR,1,"Senseman, Samuel",NCF,Northern Colorado Fencers,B2016,
2,Cadet (U16) Men's Epee,11,NR,1,"Senseman, Samuel",NCF,Northern Colorado Fencers,B2016,
3,Cadet (U16) Men's Epee,11,NR,1,"Senseman, Samuel",NCF,Northern Colorado Fencers,B2016,
4,Cadet (U16) Men's Epee,11,NR,1,"Senseman, Samuel",NCF,Northern Colorado Fencers,B2016,
...,...,...,...,...,...,...,...,...,...
252,Div 1A Senior Women's Saber,10,E1,1,"Senseman, Samuel",NCF,Northern Colorado Fencers,B2016,
253,Div 1A Senior Women's Saber,10,E1,1,"Senseman, Samuel",NCF,Northern Colorado Fencers,B2016,
254,Div 1A Senior Women's Saber,10,E1,1,"Senseman, Samuel",NCF,Northern Colorado Fencers,B2016,
255,Div 1A Senior Women's Saber,10,E1,1,"Senseman, Samuel",NCF,Northern Colorado Fencers,B2016,


In [48]:
[t.strip() for t in rows[2].split('\n') if t != '']

['1', 'Senseman, Samuel', 'NCF', 'Northern Colorado Fencers', 'B2016']

In [46]:
rows[2]

'1\n\xa0\xa0Senseman, Samuel\n\nNCF\nNorthern Colorado Fencers\n\nB2016'